In [1]:
import os
import requests
import json
from datetime import datetime
from typing import Union, List
import pandas as pd
from dotenv import load_dotenv


In [2]:
class APIRequestError(Exception):
    def __init__(self, status_code, message, function_name):
        self.status_code = status_code
        self.message = message
        self.function_name = function_name
        super().__init__(f"HTTP error {self.status_code} occurred in {self.function_name}: {self.message}")



In [23]:
#Primero: funcion para traer serie intradiaria en 60 min del stock que necesito

def intraday_stock_serie(symbol:str, interval:str):   
    function = 'TIME_SERIES_INTRADAY'
    adjusted=True
    extended_hours=False
    size = 'compact'
    parameters_market_data = {'function':function, 'symbol':symbol, 'interval':interval, 'extended_hours':extended_hours,
            'adjusted':adjusted,'outputsize':size,'apikey':token }
    try:
        r = requests.get(base_url, params=parameters_market_data)
        r.raise_for_status()  
        data = r.json()
        if "Error Message" in data:
            error_message = data["Error Message"]
            raise APIRequestError(r.status_code, error_message, "intraday_stock_serie")
        else:
            data = data[f'Time Series ({interval})']
            return data
    except requests.exceptions.HTTPError as http_err:
        raise APIRequestError(http_err.response.status_code, http_err, "intraday_stock_serie")
    except Exception as err:
        raise APIRequestError(500, str(err), "intraday_stock_serie")

In [11]:
#Segundo: funcion para traer noticias relacionadas a ese stock

def getSentiment(
    symbol: str,
    topics: Union[str, List[str]]
):
    
    # Convierto topics en un solo string si vino en una lista de strings
    if isinstance(topics, list):
        topics = ','.join(topics)
    
    parameters_news_sentiment_data = {
        'function': 'NEWS_SENTIMENT',
        'tickers': symbol,
        'topics': topics,
        'apikey': token
    }
    
    try:
        r = requests.get(base_url, params=parameters_news_sentiment_data)
        r.raise_for_status() 
        data = r.json()
        data_feed = data['feed']
        data_sentiment = []
        for i in data_feed:
            for item in i['ticker_sentiment']:
                if item['ticker'] == symbol:
                    # Formateo time_published 
                    time_published = datetime.strptime(i['time_published'], '%Y%m%dT%H%M%S')
                    formatted_time_published = time_published.strftime('%Y-%m-%d %H:%M')
                    data_sentiment.append({
                        'ticker': item['ticker'],
                        'time_published': formatted_time_published,
                        'source_domain': i['source_domain'],
                        'relevance_score': item['relevance_score'],
                        'ticker_sentiment_label': item['ticker_sentiment_label']
                    })
        return data_sentiment
    except requests.exceptions.HTTPError as http_err:
        raise APIRequestError(http_err.response.status_code, http_err, "getSentiment")
    except Exception as err:
        raise APIRequestError(500, str(err), "getSentiment")




In [12]:
# 3 Unifico las funciones de market data y news data en una sola 
def get_stock_data(tickers, interval, topics):
    stock_data_frames = {}
    
    for ticker in tickers:
        try:
            # Traigo intraday stock data
            intraday_data = intraday_stock_serie(ticker, interval)
            
            # Traigo sentiment data
            sentiment_data = getSentiment(ticker, topics)
            
            # Convierto en pandas dataframe
            df_intraday = pd.DataFrame(intraday_data).transpose()
            df_intraday.index = pd.to_datetime(df_intraday.index)
            df_sentiment = pd.DataFrame(sentiment_data)
            
            # Guardo los df en diccionarios
            stock_data_frames[ticker] = {
                'intraday_data': df_intraday,
                'sentiment_data': df_sentiment
            }
        except APIRequestError as api_err:
            print(f"{api_err.function_name}: API Request Error - Status Code {api_err.status_code}: {api_err.message}")
            # You can handle the error based on the status code here.
            # For example, you may choose to skip the stock if the error is not recoverable.
            continue
    return stock_data_frames

In [13]:
#Advantage api configuration
base_url = os.environ.get('BASE_URL')
token = os.environ.get('API_TOKEN')
tickers = ['APL','IBM']
interval = '60min'
topics = 'technology, manufacturing, financial_markets'


In [14]:
data_frames_by_ticker = get_stock_data(tickers, interval, topics)

intraday_stock_serie: API Request Error - Status Code 500: 'Time Series (60min)'
intraday_stock_serie: API Request Error - Status Code 500: 'Time Series (60min)'


In [15]:
parameters_market_data = {'function':'TIME_SERIES_INTRADAY', 'symbol':'Aasdr', 'interval':interval, 
           'outputsize':'compact','apikey':token }
r = requests.get(base_url, params=parameters_market_data)
r.raise_for_status() 
r 

<Response [200]>

In [24]:
intraday_stock_serie('IBMS','60min')

APIRequestError: HTTP error 500 occurred in intraday_stock_serie: HTTP error 200 occurred in intraday_stock_serie: Invalid API call. Please retry or visit the documentation (https://www.alphavantage.co/documentation/) for TIME_SERIES_INTRADAY.

In [19]:
import requests
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=IBMS&interval=5min&apikey=OYDV8S8KFCSH3B35'
r = requests.get(url)
data = r.json()

print(data)

{'Error Message': 'Invalid API call. Please retry or visit the documentation (https://www.alphavantage.co/documentation/) for TIME_SERIES_INTRADAY.'}


In [20]:
r = requests.get(url, params=parameters_market_data)


In [22]:
r.json()

{'Error Message': 'Invalid API call. Please retry or visit the documentation (https://www.alphavantage.co/documentation/) for TIME_SERIES_INTRADAY.'}